In [11]:
import re
from datetime import datetime

In [2]:
arcpy.env.workspace

'E:\\ESF MU\\Bikesharing\\ESF_data\\ESF_data_view\\Default.gdb'

In [3]:
fcs = arcpy.ListFeatureClasses()
fcs

['BikeSharing_XYToLine', 'MUNIeu_info_all_FeatureToPoi', 'MUNIeu_info_all_ETRS', 'BikeSharing_ETRS', 'MUNIeu_info_all_FeatureToPoi_ETRS', 'MUNIeu_info_all_Line_to_sea', 'NUTS_1_units', 'Company_Points_EU', 'EU_units_selection', 'MUNIeu_info_all_ETRS_Diss_all_LAU_C', 'MUNIeu_info_all_ETRS_Diss_all_LAU_C_Selection', 'cycleway', 'MUNIeu_info_all_LCC_Diss_all1', 'OverUnits', 'OverUnits_Merge', 'EU_diss_units', 'EU_diss_units_2', 'EU_diss_units_2_SpatialJoin_3', 'EU_diss_units_2_FeatureToPoi', 'EU_diss_units_2_SpatialJoin', 'EU_diss_units_2_SpatialJoin_SummarizeWithin', 'EU_diss_units_2_SpatialJoin_', 'EU_units_charac_SJ', 'EU_units_charac_companies_SJ', 'EU_units_charac_WGS84', 'BikeSharing_stations_LCC_Spa']

In [4]:
datasets = arcpy.ListDatasets("*",feature_type="Feature")

In [5]:
datasets

[]

In [6]:
arcpy.Exists("EU_units_charac_4")

True

In [7]:
fc = "EU_units_charac_4"
fc_flds = [f.name for f in arcpy.ListFields(fc)]
fc_flds

['OBJECTID_12', 'Shape', 'BikeShareMap_stations', 'TARGET_FID', 'all__GISCO', 'all__LAU18', 'SUM_all__pop20', 'OBJECTID', 'all__GISCO_1', 'ZONE_CODE', 'COUNT', 'AREA', 'MIN', 'MAX', 'MEAN', 'OBJECTID_1', 'all__GISCO_12', 'ZONE_CODE_1', 'COUNT_1', 'AREA_1', 'MIN_1', 'MAX_1', 'MEAN_1', 'Area_km', 'sum_shape_length', 'sum_Length_KILOMETERS', 'Polyline_Count', 'NEAR_FID', 'NEAR_DIST_m', 'NEAR_X', 'NEAR_Y', 'NEAR_ANGLE', 'Shape_Length', 'Shape_Area', 'Short_GISCO']

In [8]:
#arcpy.management.AddField(fc, "Short_GISCO", "TEXT", field_length=5, field_alias="Short_GISCO")

In [9]:
for name in fc_flds:
    idx = fc_flds.index(name)
    print(idx, name)

0 OBJECTID_12
1 Shape
2 BikeShareMap_stations
3 TARGET_FID
4 all__GISCO
5 all__LAU18
6 SUM_all__pop20
7 OBJECTID
8 all__GISCO_1
9 ZONE_CODE
10 COUNT
11 AREA
12 MIN
13 MAX
14 MEAN
15 OBJECTID_1
16 all__GISCO_12
17 ZONE_CODE_1
18 COUNT_1
19 AREA_1
20 MIN_1
21 MAX_1
22 MEAN_1
23 Area_km
24 sum_shape_length
25 sum_Length_KILOMETERS
26 Polyline_Count
27 NEAR_FID
28 NEAR_DIST_m
29 NEAR_X
30 NEAR_Y
31 NEAR_ANGLE
32 Shape_Length
33 Shape_Area
34 Short_GISCO


In [14]:
# iterování přes all__GISCO atribut a uložení do slovníku v podobě {all__GISCO:*****(prvních 5 znaků)}

id_dic = dict()

counter = 0

with arcpy.da.SearchCursor(fc, fc_flds) as cur:
    for row in cur:
        if counter < 4:
            all_GISCO = (row[4])
            all_GISCO_temp = row[4]
            #print(all_GISCO)
            new_ID = re.findall("^.{0,5}",all_GISCO_temp)
            
            #print(list(zip(all_GISCO,new_ID)))
            #id_dic = {k:v for k,v in zip(all_GISCO, new_ID)}
            id_dic[all_GISCO] = new_ID
            print(all_GISCO)
            print(new_ID)
            
            counter += 1
        else:
            print(id_dic)
            break

AT_20101
['AT_20']
AT_40101
['AT_40']
AT_50101
['AT_50']
AT_60101
['AT_60']
{'AT_20101': ['AT_20'], 'AT_40101': ['AT_40'], 'AT_50101': ['AT_50'], 'AT_60101': ['AT_60']}


In [17]:
print("Start:", datetime.now())
start_time = datetime.now()
print()

id_dic = dict()

with arcpy.da.UpdateCursor(fc, fc_flds) as cur2:
    for row2 in cur2:
        all_GISCO = row2[4]
        all_GISCO_temp = row2[4]
        new_ID = all_GISCO_temp[0:5] #re.findall("^.{0,5}",all_GISCO_temp)
        #for temp_ID in new_ID:    
        #print(new_ID)
        #break
        id_dic[all_GISCO] = new_ID
        #print(id_dic)
        
        row2[34] = id_dic[all_GISCO]
        
        cur2.updateRow(row2)
        #break
        
        
        
        
print()                    
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))
print("End:", datetime.now())

Start: 2022-07-12 01:55:50.870306


Duration: 0:00:44.326009
End: 2022-07-12 01:56:35.197312
